In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset,DataLoader
from PIL import Image
from torch.cuda.amp import GradScaler, autocast

In [ ]:
#df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')

In [ ]:
#df

In [ ]:
from tqdm import tqdm

In [ ]:
# base_path = "/kaggle/input/rsna-cut-off-empty-space-from-images"
# all_paths = []
# for k in tqdm(range(len(df))):
#     row = df.iloc[k, :]
#     all_paths.append(base_path +"/" +str(row.patient_id) + "/" + str(row.image_id) + ".png")
    
# df["path"] = all_paths

In [ ]:
# df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder, normalize

In [ ]:
le = LabelEncoder()

In [ ]:
# df['laterality'] = le.fit_transform(df['laterality'])
# df['view'] = le.fit_transform(df['view'])
# df['BIRADS'] = le.fit_transform(df['BIRADS'])
# df['density'] = le.fit_transform(df['density'])
# df['difficult_negative_case'] =le.fit_transform(df['difficult_negative_case'])

In [ ]:
#df.head()

In [ ]:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# df[['age']] = imputer.fit_transform(df[['age']])

In [ ]:
#df.isnull().sum()

In [ ]:
#df.to_csv('rsna_df.csv',index=None)

In [ ]:
train_df = pd.read_csv('/kaggle/input/rsna-datadrame/rsna_df.csv')

In [ ]:
train_df.head()

In [ ]:
ids = []
for i in range(len(train_df)):
    ids.append(int(train_df.loc[i,'path'].split('/')[-1][:-4]))

In [ ]:
train_df['cancer'].value_counts()

In [ ]:
POS = int(53548/1158)
POS

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available now:', DEVICE)

In [ ]:
import torchvision
from PIL import Image

def get_transforms(aug=False):

    def transforms(img):
#         img = img.convert('RGB')#.resize((512, 512))
        if aug:
            tfm = [
                torchvision.transforms.RandomHorizontalFlip(0.5),
                torchvision.transforms.RandomRotation(degrees=(-5, 5)), 
                torchvision.transforms.RandomResizedCrop((1024, 512), scale=(0.8, 1), ratio=(0.45, 0.55)) 
            ]
        else:
            tfm = [
                torchvision.transforms.RandomHorizontalFlip(0.5),
                torchvision.transforms.RandomVerticalFlip(0.5),
                torchvision.transforms.Resize((256,256))
            ]
        img = torchvision.transforms.Compose(tfm + [            
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            
        ])(img)
        return img
    return lambda img: transforms(img)

In [ ]:
Target = 'cancer'
class RSNADataset(Dataset):
    def __init__(self, df, transforms=None):
        super().__init__()
        self.df = df
        self.img_path = list(df['path'])
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img = self.img_path[idx]
        img = Image.open(img).convert('RGB')
        if (self.transforms!=None):
            img = self.transforms(img)
        
        if Target in self.df.columns:
            target = self.df.loc[idx,'cancer']
            return img,torch.tensor(np.array([target]))
        return img

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(train_df, test_size=0.25)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [ ]:
df_train.shape,df_val.shape

In [ ]:
train_ds = RSNADataset(df_train,get_transforms(False))
val_ds = RSNADataset(df_val,get_transforms(False))

In [ ]:
val_ds[1][0].shape

In [ ]:
train_loader = DataLoader(train_ds, batch_size=32,shuffle=True,num_workers=2,pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=64,shuffle=False,num_workers=2,pin_memory=True)

In [ ]:
img,label = next(iter(train_loader))
img.shape,label.shape

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
train_dl = DeviceDataLoader(train_loader,device)
val_dl = DeviceDataLoader(val_loader,device)

In [ ]:
import gc
gc.collect()

**BO EFFNET**

In [ ]:
!pip install timm

In [ ]:
import timm

class EfficientNet_b0(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = timm.create_model("efficientnet_b0",pretrained=True)
        # Replace last layer
        num_ftrs = self.network.classifier.in_features
#         self.network.classifier = nn.Linear(num_ftrs,3)
        self.network.classifier = torch.nn.Sequential(torch.nn.Linear(num_ftrs,1))
    
    def forward(self, xb):
        return self.network(xb)

In [ ]:
effnet_b0 = EfficientNet_b0()
effnet_b0 = effnet_b0.to(device)
model = effnet_b0

In [ ]:
def evaluate_model(model,val_dl):
    torch.manual_seed(42)
    pred_cancer = []
    with torch.no_grad():
        model.eval()
        losses = []
        targets = []
        with tqdm(val_dl,desc='Eval',mininterval=30) as eval_progress:
            for i,batch in enumerate(eval_progress):
                with autocast(enabled=True):
                    imgs,labels = batch
                    cancer_pred = model.forward(imgs)
                    loss = F.binary_cross_entropy_with_logits(
                            cancer_pred,
                            labels.to(float),
                            pos_weight = torch.tensor([20]).to(device)
                        ).item()
                    losses.append(loss)
                    pred_cancer.append(torch.sigmoid(cancer_pred))
                    targets.append(labels.cpu().numpy())
        targets = np.concatenate(targets)
        pred = torch.concat(pred_cancer).cpu().numpy()
    return np.mean(losses)

In [ ]:
loss = evaluate_model(effnet_b0,val_dl)

In [ ]:
def save_model(name, model, thres, model_type):
    torch.save({'model': model.state_dict(), 'threshold': thres, 'model_type': model_type}, f'{name}')

In [ ]:
def pfbeta(labels, predictions, beta=1.):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / max(y_true_count, 1)  # avoid / 0
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0

def optimal_f1(labels, predictions):
    thres = np.linspace(0, 1, 101)
    f1s = [pfbeta(labels, predictions > thr) for thr in thres]
    idx = np.argmax(f1s)
    return f1s[idx], thres[idx]

In [ ]:
l = torch.tensor(np.array([[0],[0],[1],[0],[1]])).squeeze()
p = torch.randn(5,1).squeeze()
p

In [ ]:
optimal_f1(l,p)

In [ ]:
def evaluate_model(model,val_dl):
    torch.manual_seed(42)
    pred_cancer = []
    with torch.no_grad():
        model.eval()
        losses = []
        targets = []
        with tqdm(val_dl,total=len(val_dl),desc='Eval') as eval_progress:
            for i,batch in enumerate(eval_progress):
                with autocast(enabled=True):
                    imgs,labels = batch
                    cancer_pred = model.forward(imgs)
                    loss = F.binary_cross_entropy_with_logits(
                            cancer_pred,
                            labels.to(float),
                            pos_weight = torch.tensor([20]).to(device)
                        ).item()
                    losses.append(loss)
                    pred_cancer.append(torch.sigmoid(cancer_pred))
                    targets.append(labels.cpu().numpy())
                mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
                eval_progress.set_postfix(val_loss=f'{loss:0.4f}',gpu_mem=f'{mem:0.2f} GB')
        targets = np.concatenate(targets)
        pred = torch.concat(pred_cancer).cpu().numpy()
        pf1,thres = optimal_f1(targets.squeeze(),pred.squeeze())
    return np.mean(losses),(pf1,thres),pred

In [ ]:
loss = evaluate_model(effnet_b0,val_dl)

In [ ]:
loss

In [ ]:
effnet_b0.parameters()

In [ ]:
def train_model(train_dl,val_dl):
    torch.manual_seed(42)
    model = EfficientNet_b0().to(device)
    optim = torch.optim.Adam(model.parameters())
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optim,max_lr=0.0004,epochs=5,steps_per_epoch = len(train_dl),pct_start = 0.1)
    name = "Effnet_B0"
    model_type = "efficientnet_b0"
    scaler = GradScaler()
    best_eval_score = 0
    for epoch in range(4):
        print(f'Epoch : {epoch}')
        model.train()
        with tqdm(train_dl,total=len(train_dl),desc='Train') as train_progress:
            for idx,batch in enumerate(train_progress):
                images,labels = batch
                optim.zero_grad()
                with autocast(enabled=True):
                    cancer_pred = model.forward(images)
                    loss = F.binary_cross_entropy_with_logits(
                            cancer_pred,
                            labels.to(float),
                            pos_weight = torch.tensor([20]).to(device)
                        )
                    if np.isinf(loss.item()) or np.isnan(loss.item()):
                                print(f'Bad Loss, skipping the batch {idx}')
                                del loss,cancer_pred
                                gc_collect()
                                continue
                scaler.scale(loss).backward()
                scaler.step(optim)
                scheduler.step()
                scaler.update()
                lr = scheduler.get_last_lr()[0]
                mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
                train_progress.set_postfix(train_loss=f'{loss:0.4f}',gpu_mem=f'{mem:0.2f} GB',lr=f'{lr:0.4f}')
                
        
        eval_loss,(f1,thres),pred = evaluate_model(model,val_dl)
        if (f1>best_eval_score):
            best_eval_score = f1
            save_model(name,model,thres,model_type)
    return model

In [ ]:
train_model(train_dl,val_dl)

In [ ]:
model = effnet_b0

In [ ]:
torch.save(model.state_dict(),"checknew.pth")